In [1]:
!pip install --upgrade pip
!pip install torch transformers sentence-transformers faiss-cpu pandas openpyxl
!pip install --upgrade ipywidgets

from tqdm.notebook import tqdm
import time

for _ in tqdm(range(10)):
    time.sleep(0.1)

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
import sqlite3, os, sys
import pandas as pd

# exit if the database already exists
if os.path.exists("YoY_filtered_V15.db"):
    print("Database already exists")
    sys.exit()

# Load the Excel file
file_path = "./YoY_filtered_V15 4.xlsx"
df = pd.read_excel(file_path)

# Add a unique ID for each record
df['ID'] = df.index

# Connect to the SQLite database
conn = sqlite3.connect("YoY_filtered_V15.db")
cursor = conn.cursor()

# Create the `RiskRecords` table
cursor.execute('''
CREATE TABLE IF NOT EXISTS risk_records (
    ID INTEGER PRIMARY KEY,
    Risk_Category TEXT,
    Risk_Factor TEXT,
    Filing_Year INTEGER,
    Filing_Link TEXT,
    Company_Name TEXT,
    Ticker TEXT,
    Risk_Detail TEXT,
    DUNS INTEGER,
    Sector TEXT,
    Industry TEXT,
    CIK INTEGER,
    Topic TEXT,
    Update_Status TEXT,
    Position INTEGER,
    Position_Change INTEGER,
    Number_of_Risk_Factors INTEGER,
    Previous_Number_of_Risk_Factors INTEGER,
    CGI_ID INTEGER,
    Year_Change_Summary TEXT,
    Peer_Group_ID INTEGER,
    Peer_Group_Name TEXT,
    Risk_ID TEXT,
    Topic_number INTEGER,
    Topic_Tier_1 TEXT
)
''')

# Insert records from Excel
df.to_sql("records", conn, if_exists="replace", index=False)

# Create the `embeddings` table
cursor.execute('''
CREATE TABLE IF NOT EXISTS embeddings (
    ID INTEGER PRIMARY KEY,
    embedding BLOB,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY(ID) REFERENCES records(ID)    
)
''')

conn.commit()

Database already exists


: 

In [4]:
import datetime
import sqlite3
import torch
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm
import pandas as pd

def get_records_to_update(cursor, age_hours=24):
    """Get records that need their embeddings updated"""
    cutoff_time = datetime.now() - timedelta(hours=age_hours)
    
    cursor.execute('''
    SELECT r.*, e.updated_at 
    FROM records r 
    LEFT JOIN embeddings e ON r.ID = e.ID 
    WHERE e.updated_at IS NULL 
    OR e.updated_at < ?
    ''', (cutoff_time,))
    
    columns = [description[0] for description in cursor.description]
    records = cursor.fetchall()
    
    # Convert to DataFrame with column names
    return pd.DataFrame(records, columns=columns)

def update_embeddings(df, model):
    """Update embeddings for the given records"""
    current_time = datetime.now().isoformat()
    
    # Combine fields - using actual column names from database
    texts = df.apply(lambda x: f"Sector:{x['Sector']}\n"
                             f"Industry:{x['Industry']}\n"
                             f"Topic:{x['Topic']}\n"
                             f"Risk Category:{x['Risk Category']}\n"
                             f"Risk Factor:{x['Risk Factor']}\n"
                             f"Risk Detail:{x['Risk Detail']}\n"
                             f"Company Name:{x['Company Name']}", axis=1)
    
    # Generate embeddings in batches
    batch_size = 32  # Smaller batch size to avoid memory issues
    embeddings = []
    
    # Use model.device instead of _target_device
    device = model.device
    
    for i in tqdm(range(0, len(texts), batch_size), desc="Generating embeddings"):
        batch_texts = texts[i:i + batch_size].tolist()
        batch_embeddings = model.encode(batch_texts, 
                                       convert_to_tensor=True, 
                                       device=device)
        embeddings.extend(batch_embeddings.cpu().numpy())
    
    # Update database
    conn = sqlite3.connect("YoY_filtered_V15.db")
    cursor = conn.cursor()
    
    for id, emb in zip(df['ID'], embeddings):
        cursor.execute(
            "INSERT OR REPLACE INTO embeddings (ID, embedding, updated_at) VALUES (?, ?, ?)",
            (int(id), emb.tobytes(), current_time)
        )
        
    conn.commit()
    conn.close()
    
# Initialize model with device handling
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SentenceTransformer('all-MiniLM-L6-v2').to(device)

# Get records that need updating
conn = sqlite3.connect("YoY_filtered_V15.db")
records_df = get_records_to_update(cursor)

if len(records_df) > 0:
    print(f"Updating {len(records_df)} records...")
    update_embeddings(records_df, model)
else:
    print("No records need updating")

conn.close()


/var/folders/7z/d0fwmcf95t9dssllh1bsczjc0000gp/T/ipykernel_36990/3092861322.py:14: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.execute('''


Updating 123721 records...
fields: Index(['Risk Category', 'Risk Factor', 'Filing Year', 'Filing Link',
       'Company Name', 'Ticker', 'Risk Detail', 'DUNS', 'Sector', 'Industry',
       'CIK', 'Topic', 'Update Status', 'Position', 'Position Change',
       'Number of Risk Factors', 'Previous Number of Risk Factors', 'CGI_ID',
       'Year Change Summary', 'Peer Group ID', 'Peer Group Name', 'Risk_ID',
       'Topic_number', 'Topic Tier 1', 'ID', 'updated_at'],
      dtype='object')


`SentenceTransformer._target_device` has been deprecated, please use `SentenceTransformer.device` instead.


Batches:   0%|          | 0/3867 [00:00<?, ?it/s]

KeyboardInterrupt: 

KeyboardInterrupt: 